In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import preprocessing 
from sklearn.metrics import classification_report

data = pd.read_csv('nbaHomeWinLossModelDataset.csv').drop(['Unnamed: 0'],axis=1)
data = data.dropna() 
data[data.isnull().any(axis=1)]
data.head(10) 


In [ ]:
test = data[data["SEASON"] == ("2020-21" or "2021-22")].sample(frac=1) 
train = data[data['SEASON'] == '2022-23']

test1 = data[data["SEASON"] == ("2021-22")].sample(frac=1) 
train1 = data[data['SEASON'] == '2022-23']

X = test.drop(['HOME_W','SEASON'],axis=1)
y = test['HOME_W']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.33)

X1 = test1.drop(['HOME_W','SEASON'],axis=1)
y1 = test1['HOME_W']
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,test_size=.33) 

# Standard Scaling Prediction Variables
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
scaled_data_train = scaler.transform(X_train)
scaled_data_test = scaler.transform(X_test)

# Standard Scaling Prediction Variables
scaler1 = preprocessing.StandardScaler()
scaler1.fit(X1_train)
scaled_data_train1 = scaler.transform(X1_train)
scaled_data_test1 = scaler.transform(X1_test)

print("Shape of Training and Testing Data for 20-22 Training Data")
print(train.shape) 
print(test.shape)

print("Shape of Training and Testing Data for 21-22 Training Data")
print(train1.shape) 
print(test1.shape)

In [ ]:
#Logistic Regression
model = LogisticRegression()
model.fit(scaled_data_train,y_train)
model.score(scaled_data_test,y_test)

F1Score = cross_val_score(model,scaled_data_test,y_test,cv=12,scoring='f1_macro');
y_pred = model.predict(scaled_data_test)
print("Analysis for Model trained on the 2020-2021 and 2021-2022 Season Data\n")
print("20-22 Logistic Model F1 Accuracy: %0.2f (+/- %0.2f)"%(F1Score.mean(), F1Score.std() *2))
print("Classification Report for Testing Data From the Training Split")
print(classification_report(y_test,y_pred))

# Standard Scaling Prediction Variables
scaler = preprocessing.StandardScaler()
scaler.fit(test.drop(['HOME_W','SEASON'],axis=1))
scaled_val_data = scaler.transform(test.drop(['HOME_W','SEASON'],axis=1))

# How the model performs on unseen data
y_pred = model.predict(scaled_val_data)
print("Classification Report for Predictions on the 2022-2023 Season")
print(classification_report(test['HOME_W'],y_pred)) 
print("###################################################################") 

#Logistic Regression
model1 = LogisticRegression()
model1.fit(scaled_data_train1,y1_train)
model1.score(scaled_data_test1,y1_test)

F1Score1 = cross_val_score(model1,scaled_data_test1,y1_test,cv=12,scoring='f1_macro');
y1_pred = model1.predict(scaled_data_test1)
print("Analysis for Model trained on the 2021-2022 Season Data\n")
print("21-22 Logistic Model F1 Accuracy: %0.2f (+/- %0.2f)"%(F1Score1.mean(), F1Score1.std() *2))
print("Classification report for Testing Data From the Training Split")
print(classification_report(y1_test,y1_pred))

# Standard Scaling Prediction Variables
scaler1 = preprocessing.StandardScaler()
scaler1.fit(test1.drop(['HOME_W','SEASON'],axis=1))
scaled_val_data1 = scaler1.transform(test1.drop(['HOME_W','SEASON'],axis=1))

# How the model performs on unseen data
y1_pred = model1.predict(scaled_val_data1) 
print("Classification Report for predictions on 2022-2023 Season")
print(classification_report(test1['HOME_W'],y1_pred)) 
